## SquamataGravity - Jupyter notebook for releasing Gravity data to ScienceBase

### Version 1.0

This module performs the following operations:
- Creates Entity and Attribute text for Gravity Data shapefile.
- Identify and load Gravity *.CSV data file.
- Create file list of data release files

Known issues needing repair:
- Add items here...

### Instructions
- Load Gravity CSV file.
- Create Shape file Entity and Attribute Text
- Cope text into Gravity Metadata.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.0: Started by modifying SquamaterAeroMagv1-4


In [1]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [3]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
import re
import datetime
import dateutil.parser
import time

# 1) Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- gravCSVPath: This is the path to the gravity data
- gravCSVName: The name of the gravity csv file
- gravShapefileNameLessFileExtension: the name format for the shape file.  This name should not have an extension.

In [15]:
#Set Data Paths and file names
gravCSVPath = r"C:\DataReleases\IowaGravity"
gravCSVName = "Iowa_gravity_data_release.csv"
gravShapefileNameLessFileExtension = "Iowa_gravity2019"

In [19]:
#Check Paths for the fun of it
print ('The Gravity CSV Data Path is: ' + '"' + gravCSVPath + '"')
csvFilePath = ''
csvFilePath = gravCSVPath + "\\" + gravCSVName
print ('The Gravity CSV file is: ' + csvFilePath)

The Gravity CSV Data Path is: "C:\DataReleases\IowaGravity\"
The Gravity CSV file is: C:\DataReleases\IowaGravity\\Iowa_gravity_data_release.csv


## Lets now import and index values from the CSV file
- We need create arrays of description and unit values for the metadata template.  
- We also want to run stats on the csv columns for the entity and attributes section

In [20]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.

gravDataFrame = pd.read_csv(csvFilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
gravDataFrame

,station,lat_NAD83,long_NAD83,elev_NAVD88,ObsGrav,TheoGrav,FAC,BSC,FAA,SBA,ITC,OTC,CBA
0,MA001,42.40421,-91.28302,271.452,980296.122,980384.551,83.770,30.375,-4.659,-35.034,0.015,0.030,-34.989
1,MA002,42.37231,-91.23874,326.485,980278.145,980381.676,100.753,36.534,-2.778,-39.312,0.106,0.188,-39.017
2,MA003,42.35511,-91.23856,278.489,980284.151,980380.127,85.942,31.163,-10.034,-41.197,0.038,0.023,-41.136
3,MA004,42.32513,-91.23831,277.433,980277.342,980377.426,85.616,31.045,-14.468,-45.513,0.026,0.020,-45.467
4,MA005,42.30882,-91.23939,281.886,980273.082,980375.958,86.990,31.543,-15.886,-47.429,0.088,0.030,-47.311
5,MA006,42.34718,-91.20912,295.193,980277.797,980379.413,91.097,33.032,-10.520,-43.552,0.017,0.045,-43.490
6,MA007,42.36488,-91.18992,296.155,980281.894,980381.007,91.394,33.140,-7.720,-40.859,0.012,0.043,-40.804
7,MA008,42.37986,-91.21450,271.848,980290.376,980382.357,83.892,30.420,-8.090,-38.509,0.014,0.018,-38.477
8,MA009,42.40465,-91.18860,275.307,980295.873,980384.591,84.960,30.807,-3.758,-34.565,0.020,0.016,-34.529
9,MA010,42.40472,-91.22415,274.815,980294.097,980384.597,84.808,30.752,-5.692,-36.443,0.030,0.026,-36.388


In [21]:
# now get an array of the header names
gravHeaderList = list(gravDataFrame.columns.values)
gravHeaderList

['station',
 'lat_NAD83',
 'long_NAD83',
 'elev_NAVD88',
 'ObsGrav',
 'TheoGrav',
 'FAC',
 'BSC',
 'FAA',
 'SBA',
 'ITC',
 'OTC',
 'CBA']

In [23]:
#Create Entity and Attribute Text:
csvAttributes = ''
csvEntityDescription = []
csvEntityUnit = []
csvEntityDescription.append("Station")#station
csvEntityUnit.append("UDOM")#station
csvEntityDescription.append("Base A Total Magnetic Field; mag base station, edited")#lat_NAD83
csvEntityUnit.append("nanoteslas")#lat_NAD83
csvEntityDescription.append("Direction (Degrees from North)")#long_NAD83
csvEntityUnit.append("degrees")#long_NAD83
csvEntityDescription.append("Fiducial Time, UTC seconds past midnight")#elev_NAVD88
csvEntityUnit.append("seconds")#elev_NAVD88
csvEntityDescription.append("Time")#ObsGrav
csvEntityUnit.append("hours")#ObsGrav
csvEntityDescription.append("Year and Julian Date")#TheoGrav
csvEntityUnit.append("YYYY.DDDDDD")#TheoGrav
csvEntityDescription.append("Latitude, WGS84")#FAC
csvEntityUnit.append("degrees")#FAC
csvEntityDescription.append("Line Number")#BSC
csvEntityUnit.append("none")#BSC
csvEntityDescription.append("Longitude, WGS84")#FAA
csvEntityUnit.append("degrees")#FAA
csvEntityDescription.append("Total Magnetic Field, lag removed")#SBA
csvEntityUnit.append("nanoteslas")#SBA
csvEntityDescription.append("Applied IGRF Field")#ITC
csvEntityUnit.append("nanoteslas")#ITC
csvEntityDescription.append("Total Magnetic Field, partial IGRF removed; partial IGRF to drape surface removed from m3l")#OTC
csvEntityUnit.append("nanoteslas")#OTC
csvEntityDescription.append("Total Magnetic Field, diurnals corrected;filtered diurnal signal removed from mreslc")#CBA
csvEntityUnit.append("nanoteslas")#CBA
csv
#Now get a min and max value for all columns
csvMax = []
csvMin = []
entAttText = ""
csvHeaderListLength = len(gravHeaderList)
for i in range(csvHeaderListLength):
 #print (magHeaderList[i])
 strCol = gravHeaderList[i]
 csvMax.append(gravDataFrame[strCol].max())
 csvMin.append(gravDataFrame[strCol].min())
 print ('Max. Value ' + gravHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + gravHeaderList[i] + ' :' + str(csvMin[i]))
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + csvEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + csvEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 entAttText = entAttText + entAtti

csvAttributes = entAttText

print ('\nEntity and Attribute Text:\n' + csvAttributes)
 

Max. Value station :NEI808
Min. Value station :MA001
Max. Value lat_NAD83 :42.99592
Min. Value lat_NAD83 :41.84599
Max. Value long_NAD83 :-91.16644000000001
Min. Value long_NAD83 :-92.49545
Max. Value elev_NAVD88 :380.462
Min. Value elev_NAVD88 :196.12900000000002
Max. Value ObsGrav :980388.762
Min. Value ObsGrav :980239.599
Max. Value TheoGrav :980437.915
Min. Value TheoGrav :980334.303
Max. Value FAC :117.411
Min. Value FAC :60.526
Max. Value BSC :42.574
Min. Value BSC :21.947
Max. Value FAA :44.135
Min. Value FAA :-38.135999999999996
Max. Value SBA :8.113999999999999
Min. Value SBA :-71.842
Max. Value ITC :1.031
Min. Value ITC :0.0
Max. Value OTC :0.45
Min. Value OTC :-0.052000000000000005
Max. Value CBA :8.162
Min. Value CBA :-71.822

Entity and Attribute Text:

				<attr>
					<attrlabl>station</attrlabl>
					<attrdef>Station</attrdef>
					<attrdefs>EON GEOSCIENCES INC.</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>MA001</rdommin>
							<rdommax>NEI808</rdommax>
					

In [20]:
#strip Tabs from string
stripFileListing = fileListing.replace('\t', '')
#Create array of values by splitting on line return \n
arrayFileListing = stripFileListing.splitlines()
arrayFileListing

['11003_VillaGrove_Final_Data.gdb.xml',
 '11003_VillaGrove_Final_Data_NAVD88.gdb.xml',
 '11003_VillaGrove_DTM.grd',
 '11003_VillaGrove_RadarAlt.grd',
 '11003_VillaGrove_TMF.grd',
 '11003_VillaGrove_DTM.gxf',
 '11003_VillaGrove_RadarAlt.gxf',
 '11003_VillaGrove_TMF.gxf',
 '11003_VillaGrove_Final_Data_ReadMe.rtf',
 '11003_VillaGrove_Final_Survey_Report.pdf']

In [21]:
# Create the Entity and Attributes for the other files being released besides the *.CSV data file
bFilesEntAttText = ''
bFilesEandA = ''
for i in range(len(arrayFileListing)):
    if arrayFileListing[i].find('.pdf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Contractors report in Adobe Portable Document Format (PDF).  This report describes an aeromagnetic survey over the Village Grove region of Colorado area under contract to the EON Geosciences Inc.</enttypd>' \
            + '\n\t\t\t\t<enttypds>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>report</attrlabl>' \
            + '\n\t\t\t\t<attrdef>Contractor report in Adobe PDF Format</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>This report describes the survey parameters, field operations, quality control results, and data reduction procedures used to produce the aeromagnetic data.</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if arrayFileListing[i].find('.rtf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Rich Text File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</enttypd>' \
            + '\n\t\t\t\t<enttypds>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Rich Text File</attrlabl>\n\t\t\t\t<attrdef>ReadMe file in Rich Text Format (RTF) containing *.CSV file entities, attributes, and processing notes</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if arrayFileListing[i].find('.gdb') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary data base file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary data base file</attrlabl>\n\t\t\t\t<attrdef>Binary data base file described in contractor report and ReadMe file</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Binary Data Base File described in contractor report and ReadMe file</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.xml') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII XML File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII text XML formatted metadata file describing the data base file of the same name.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII XML formatted text file</attrlabl>\n\t\t\t\t<attrdef>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.gxf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII grid file in Geosoft (https://www.geosoft.com) Grid Exchange Format, GXF (https://www.geosoft.com/media/uploads/resources/technical-notes/gxfr3d9_1.pdf) </enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII Grid File</attrlabl>\n\t\t\t\t<attrdef>ASCII grid file in GXF format described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>ASCII Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
    if arrayFileListing[i].find('.grd') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary grid file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary grid file</attrlabl>\n\t\t\t\t<attrdef>Binary grid file described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Binary Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    bFilesEntAttText = bFilesEandA
print (bFilesEntAttText) 


		<detailed>
			<enttyp>
				<enttypl>Binary File 11003_VillaGrove_Final_Data.gdb.xml</enttypl>
				<enttypd>Binary data base file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>
				<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Binary data base file</attrlabl>
				<attrdef>Binary data base file described in contractor report and ReadMe file</attrdef>
				<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>
				<attrdomv>
					<udom>Binary Data Base File described in contractor report and ReadMe file</udom>
				</attrdomv>
			</attr>
		</detailed>
		<detailed>
			<enttyp>
				<enttypl>ASCII XML File 11003_VillaGrove_Final_Data.gdb.xml</enttypl>
				<enttypd>ASCII text XML formatted metadata file describing the data base file of the same

# Populate Metadata Template

In [22]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(magMetaDataTemplatePath, magMetaDataTemplateName)
print ('Metadata path: ' + magMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\VillageGrove

['ï»¿<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '\t\t\t\t<origin>Brown, P. J.</origin>\n', '\t\t\t\t<origin>Grauch, V. J.</origin>\n', '\t\t\t\t<pubdate>2019</pubdate>\n', '\t\t\t\t<title>High Resolution Aeromagnetic Survey, Villa Grove, Colorado, USA 2012</title>\n', '\t\t\t\t<geoform>Binary Files and ASCII text</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators: Brown, P.J., http://orcid.org/0000-0002-2415-7462; Grauch, V. J., https://orcid.org/0000-0002-0761-3489</othercit>\n', '\t\t\t\t<onlink>http://dx.doi.org/10.5066/F7416V52</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', "\t\t\t<abstract>This data release includes the airborne magnetic survey data co

In [23]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
newMetaDataContent = metaDataContent
myfilename = magMetaDataTemplatePath + '\VillageGrove-Colorado-USA-AirborneMagnetics-2012.xml'
xmlFile = open(myfilename,"w+")
print(myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    {fileListing}
    if lineString.find('{fileListing}'):
     lineString = lineString.replace('{fileListing}', fileListing)
    
    if lineString.find('{csvAttributes}'):
     lineString = lineString.replace('{csvAttributes}', csvAttributes)
    
    if lineString.find('{bFilesEntAttText}'):
     lineString = lineString.replace('{bFilesEntAttText}', bFilesEntAttText)
    
    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    print (lineString)

xmlFile.close()
print ('Creation of new metadata file is complete\n\n') 


C:\CurrentWork\DataReleases\VillageGrove\VillageGrove-Colorado-USA-AirborneMagnetics-2012.xml
ï»¿<?xml version="1.0" encoding="UTF-8"?>

<metadata>

	<idinfo>

		<citation>

			<citeinfo>

				<origin>Brown, P. J.</origin>

				<origin>Grauch, V. J.</origin>

				<pubdate>2019</pubdate>

				<title>High Resolution Aeromagnetic Survey, Villa Grove, Colorado, USA 2012</title>

				<geoform>Binary Files and ASCII text</geoform>

				<pubinfo>

					<pubplace>Denver, CO</pubplace>

					<publish>U.S. Geological Survey</publish>

				</pubinfo>

				<othercit>Additional information about Originators: Brown, P.J., http://orcid.org/0000-0002-2415-7462; Grauch, V. J., https://orcid.org/0000-0002-0761-3489</othercit>

				<onlink>http://dx.doi.org/10.5066/F7416V52</onlink>

			</citeinfo>

		</citation>

		<descript>

			<abstract>This data release includes the airborne magnetic survey data collected from the Manchester region of Iowa. The Mineral Resources Program of the U.S. Geological Survey

### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/

In [ ]:
# Show the resulting child xml metadata file example 
#for i in range(len(newMetaDataContent)):
print ('Station ' + mtDataDirList[i] + ' complete')
print (newMetaDataContent)